In [ ]:
# Required libraries
%pip install matplotlib
%pip install seaborn
%pip install pandas
%pip install numpy
%pip install pymongo
%pip install psycopg2

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pymongo import MongoClient
import psycopg2

In [ ]:
# Database connections

# MongoDB
mongoClient = MongoClient("localhost", 27017) 

# PostgreSQL
postgresConn = psycopg2.connect("dbname=ium_tweb user=postgres password=admin")

In [ ]:
# Mongo tables
soccerDb = mongoClient.soccer
soccerDb.list_collection_names()

In [ ]:
# PostgreSQL tables
sqlCursor = postgresConn.cursor()
sqlCursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")
sqlCursor.fetchall()

In [ ]:
# This plot explores the relationship between the average number of goals per game and the market value of players in attacking roles

# MongoDB data gathering
cursor = soccerDb.appearances.aggregate([
    {
        "$match": {
            "$expr": {
                "$eq": [{ "$year": "$date" }, 2023]
            }
        }
    },
    {
        "$group": {
            "_id": "$player_id",
            "avgGoals": { "$avg": "$goals" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "player_id": "$_id",
            "avgGoals": { "$round": ["$avgGoals", 4] }
        }
    },
    {
        "$sort": {"avgGoals": -1}
    }
])
avgGoalsPlayer = pd.DataFrame(list(cursor))

# PostgreSQL data gathering
sqlCursor.execute("SELECT player_id, market_value_in_eur FROM player WHERE position = 'Attack'")
marketValuePlayer = pd.DataFrame(sqlCursor.fetchall())
marketValuePlayer = marketValuePlayer.rename(columns={0: "player_id", 1: "market_value"})

# Grouping the data together
valueGoalPlot = pd.merge(avgGoalsPlayer, marketValuePlayer, on="player_id")

# Cleaning from dirty data
valueGoalPlot = valueGoalPlot.dropna()

# Flattening of values
valueGoalPlot['market_value'] = np.log2(valueGoalPlot['market_value'])

In [ ]:
# Creating scatter plot
plt.figure(figsize=(12, 8))
sns.scatterplot(x=valueGoalPlot["avgGoals"], y=valueGoalPlot["market_value"], data=valueGoalPlot, hue=valueGoalPlot["avgGoals"], palette="rocket")
sns.kdeplot(x=valueGoalPlot["avgGoals"], y=valueGoalPlot["market_value"], levels=5, color="black", linewidths=1)

# Customize the plot
plt.title("Average goals vs market value for attackers")
plt.xlabel("Average goals")
plt.ylabel("Market value Log2")
plt.show()

valueGoalPlot[["market_value", "avgGoals"]].corr()

In [ ]:
# This plot explores the most valuable clubs in 2023

# PostgreSQL data gathering
sqlCursor.execute("""
                    SELECT c.name, SUM(p.market_value_in_eur), count(*) FROM club AS c 
                    INNER JOIN player AS p ON c.club_id = p.current_club_id 
                    WHERE c.last_season = p.last_season
                    GROUP BY c.name
                  """)
marketValueClub = pd.DataFrame(sqlCursor.fetchall())
marketValueClub = marketValueClub.rename(columns={0: "club_name", 1: "market_value", 2: "player_count"})

# Cleaning from dirty data
marketValueClub = marketValueClub.dropna()
marketValueClub = marketValueClub.sort_values(by=["market_value"], ascending=False)
marketValueClubTop = marketValueClub[:15].sort_values(by=["market_value"])

In [ ]:
# Creating bar plot
colors = plt.cm.viridis(np.linspace(0, 1, len(marketValueClubTop)))
plt.figure(figsize=(12, 8))
plt.barh(marketValueClubTop["club_name"], marketValueClubTop["market_value"], color=colors)

# Customize the plot
plt.title("Most valuable clubs")
plt.xlabel("Club value in bilion")
plt.ylabel("Club name")
plt.show()

In [ ]:
# This plot examines the average age of players in an Italian club

# PostgreSQL data gathering
sqlCursor.execute("""
                  SELECT c.name, p.name, DATE_PART('year', AGE(date_of_birth)) AS age, c.domestic_competition_id FROM club AS c 
                  INNER JOIN player AS p ON c.club_id = p.current_club_id 
                  WHERE c.last_season = p.last_season
                  ORDER BY c.club_id
                  """)
agePlayerClub = pd.DataFrame(sqlCursor.fetchall())
agePlayerClub = agePlayerClub.rename(columns={0: "club_name", 1: "player_name", 2: "player_age", 3: "competition"})

In [ ]:
# Creating box plot
italianClubs = agePlayerClub[agePlayerClub["competition"] == "IT1"];

# Sort data by age
sorted_data = italianClubs.groupby("club_name")["player_age"].median().sort_values()

plt.figure(figsize=(20, 8))
sns.boxplot(x="club_name", y="player_age", data=italianClubs, order=sorted_data.index, palette="light:b", legend=False)
plt.xticks(rotation=90)

# Customize the plot
plt.title("Player age in clubs")
plt.xlabel("Club name")
plt.ylabel("Player age")
plt.show()

In [ ]:
# This plot examines the average age of players in an Italian club

# PostgreSQL data gathering
sqlCursor.execute("""
                    SELECT c.name, SUM(p.market_value_in_eur) AS somma, c.stadium_name, c.stadium_seats FROM club AS c 
                    INNER JOIN player AS p ON c.club_id = p.current_club_id 
                    WHERE c.last_season = p.last_season
                    GROUP BY c.name, c.stadium_name, c.stadium_seats
                    ORDER BY somma DESC
                  """)
stadiumClubValue = pd.DataFrame(sqlCursor.fetchall())
stadiumClubValue = stadiumClubValue.rename(columns={0: "club_name", 1: "club_value", 2: "stadium_name", 3: "stadium_seats"})

# Cleaning from dirty data
stadiumClubValue = stadiumClubValue.dropna()

# Flattening of values
stadiumClubValue['club_value'] = pd.to_numeric(stadiumClubValue['club_value'], errors='coerce')
stadiumClubValue['club_value'] = np.log2(stadiumClubValue['club_value'])

In [ ]:
# Creating scatter plot
plt.figure(figsize=(10, 6))
scatter_plot = sns.scatterplot(x="stadium_seats", y="club_value", data=stadiumClubValue, hue="stadium_seats", palette="viridis", s=100, legend=False)

# Customize the plot
plt.title("Stadium Seats vs Club Value")
plt.xlabel("Stadium Seats")
plt.ylabel("Club Value Log2")
plt.show()

stadiumClubValue[["club_value", "stadium_seats"]].corr()

In [ ]:
# This plot examines the average composition of roles in a club

# PostgreSQL data gathering
sqlCursor.execute("""
                    SELECT c.name, p.position, COUNT(*) FROM club AS c 
                    INNER JOIN player AS p ON c.club_id = p.current_club_id 
                    WHERE c.last_season = p.last_season
                    GROUP BY c.name, p.position
                    ORDER BY c.name
                  """)
clubComposition = pd.DataFrame(sqlCursor.fetchall())
clubComposition = clubComposition.rename(columns={0: "club_name", 1: "player_position", 2: "occurrence"})

# Cleaning from dirty data
clubComposition = clubComposition[clubComposition["player_position"] != "Missing"]

In [ ]:
# Create a fig with 2 plots
fig, axes = plt.subplots(1, 2, figsize=(15, 7), facecolor='white')

# Creating pie plot for the left
juventusData = clubComposition[clubComposition["club_name"] == "Juventus Football Club"]
juventusData = juventusData.sort_values(by="player_position")

axes[0].pie(juventusData["occurrence"], labels=None, autopct='%1.1f%%')
axes[0].set_ylabel('')
axes[0].set_title("Composition of Juventus")

# Creating pie plot for the right
globalComposition = clubComposition[["player_position", "occurrence"]].groupby("player_position").sum().reset_index()
globalComposition = globalComposition.sort_values(by="player_position")

axes[1].pie(globalComposition["occurrence"], labels=None, autopct='%1.1f%%')
axes[1].set_ylabel('')
axes[1].set_title("Average composition of clubs")

# Legend
axes[0].legend(juventusData["player_position"], title="Player Position", loc="upper left", bbox_to_anchor=(1, 1))

In [ ]:
# Closing database connections
mongoClient.close()
postgresConn.close()